Crie um sistema capaz de resolver o problema das N-Rainhas através de busca
de melhoria iterativa (hill climbing ou têmpera simulada) para N com os seguintes
valores 10, 20 e 25. Tabele os tempos de processamento para obter uma solução.

In [107]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import colors
import re
import random

In [3]:
# #change file name
# path = "puzzle.sav"
# # -2: espaco vazio, -1: parede, 0 a 4: n de lampadas que podem iluminar, 5: lampada, 6: blocos iluminados 
# def load_puzzle(path):
#     file = open(path, "r")
#     desc = []
#     numered_block = {}
#     for line in file:
#         text = line.strip().split(":")
#         if text[0].strip() == "DESC":
#             desc = re.findall('.', text[2].strip())
#     grid = np.zeros((7, 7), dtype=int)
#     i = 0
#     run = 0
#     for x in range(7):
#         for y in range(7):
#             c = '\0'

#             if run == 0:
#                 c = desc[i]
#                 i = i + 1
#                 if c >= 'a' and c <= 'z':
#                     run = ord(c) - ord('a') + 1

#             if run > 0:
#                 c = 'S'
#                 run = run - 1

#             if c == '0' or c == '1' or c == '2' or c == '3' or c == '4':
#                 grid[x, y] = ord(c) - ord('0')
#                 numered_block[(x, y)] = grid[x, y]
#             elif c == 'B':
#                 grid[x, y] = -1
#             elif c == 'S':
#                 grid[x, y] = -2
#     return grid, numered_block

In [4]:
# path = 'puzzle.sav'
# # -2: espaco vazio, -1: parede, 0 a 4: n de lampadas que podem iluminar, 5: lampada, 6: blocos iluminados  
# grid, num_blocks = load_puzzle(path)
# print(grid)
# print(num_blocks)

In [30]:
grid = np.zeros((10, 10), dtype=int)
print(grid)

[[0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]]


In [574]:
# Modifica o grid com base no vetor de estados
# def set_state(state):

# Calcula o score do estado atual com base no número de rainhas que se atacam
# Score = Max - pares de raínhas que se atacam
def score_state(state):
    L = len(state)
    max_atacks = L*(L-1)/2
    state = np.insert(state, 0, 0, axis=0)
    score = 0
    for i in range(1, L+1):
        if i <= L:
            for j in range(i+1, L+1):
                if state[j] == state[i]:
                    score += 1
                elif abs(i - state[i]) == abs(j - state[j]):
                    score += 1
                elif state[i] + i == state[j] + j:
                    score += 1
    
    return max_atacks - score

def fitness_function(population):
    total_score = 0
    L = len(population)
    for person in population:
        total_score += score_state(person)
    
    if total_score == 0:
        return np.random.dirichlet(np.ones(L),size=1)[0]
    
    result = np.array([])
    for i in range(L):
        result = np.append(result, score_state(population[i])/total_score)
    
    return result

def selection(population, p):
    # Seleção usando o método da roleta. De toda população selecionar uma fração p
    L = len(population)
    selected = np.random.choice(np.arange(0, L, 1), int(len(population)*p), p=fitness_function(population), replace=False)
    
    result = [population[i] for i in selected]
            
    return result
        
def cross_over(population):
    selected = np.zeros(len(population)) #1: population[i] já selecionado 0: ainda não selecionado
    new_population = []
    for i in range(len(selected) - 1):
        if selected[i] != 1:
            #Seleciona esse indivíduo
            selected[i] = 1

            #Seleciona outro, à frente no vetor, aleatoriamente
            j = np.random.choice(len(selected))
            while ~(selected[j] == 0 or j <= i):
                j = np.random.choice(len(selected))

            selected[j] = 1
            
            p1 = population[i]
            p2 = population[j]
    
            #Seleciona ao acaso uma posição para o cross
            x = np.random.choice(len(population[0])) + 1

            tmp = p2[:x].copy()
            p2[:x], p1[:x]  = p1[:x], tmp

            new_population.append(p1)
            new_population.append(p2)
        
    
    return new_population
    

In [592]:
cross_over(population)

[array([1, 2, 2, 4]),
 array([1, 3, 1, 2]),
 array([1, 1, 3, 1]),
 array([1, 3, 2, 4])]

In [573]:
population = np.array([[1, 2, 3, 4], [1, 3, 2, 2], [1, 3, 2, 4], [1, 1, 1, 1]])

In [559]:
selected = np.zeros(len(population)) 
selected

array([0., 0., 0., 0.])

In [568]:
for i in range(len(selected) - 1):
    if selected[i] != 1:
        #Seleciona esse indivíduo
        selected[i] = 1

        #Seleciona outro a frente no vetor aleatoriamente
        print("i: ", i)
        j = np.random.choice(len(selected))
        while ~(selected[j] == 0 or j <= i):
            j = np.random.choice(len(selected))

        selected[j] = 1
        
selected = np.zeros(len(population)) 
 

i:  0
Esse:  [1 2 3 4] Outro:  [1 3 2 4]
i:  1
Esse:  [1 3 2 2] Outro:  [1 1 1 1]


In [486]:
np.random.choice(2)

1

In [459]:
new_population = cross_over(population)

IndexError: index 1 is out of bounds for axis 0 with size 1

In [460]:
new_population

NameError: name 'new_population' is not defined

In [461]:
unselected 

array([1])

In [ ]:
cross_over

In [415]:
p1 = [1, 1, 1, 1, 1]
p2 = [0, 0, 0, 0, 0]

In [408]:
x = np.random.choice(len(population[0])) + 1

In [416]:
x = 1

In [417]:
tmp = p2[:x].copy()
p2[:x], p1[:x]  = p1[:x], tmp

In [418]:
p2

[1, 0, 0, 0, 0]

In [419]:
p1

[0, 1, 1, 1, 1]

In [340]:
arr = [1, 2, 3]
np.delete(arr, 1, 0)

array([1, 3])

In [386]:
np.random.choice(len(population[0]))

1

In [364]:
np.arange(5) 

array([0, 1, 2, 3, 4])

In [363]:
population[0]

array([1, 2, 3, 4])

In [277]:
population = np.array([[1, 2, 3, 4], [1, 3, 2, 2], [1, 3, 2, 4]])

In [278]:
fitness_function(population)

array([0. , 0.5, 0.5])

In [321]:
np.random.choice(np.arange(0, len(population), 1), int(len(population)*0.9), p=fitness_function(population), replace=False)

array([1, 2])

In [279]:
population

array([[1, 2, 3, 4],
       [1, 3, 2, 2],
       [1, 3, 2, 4]])

In [312]:
selection(population, 0.9)

[array([1, 3, 2, 2]), array([1, 3, 2, 4])]